# Library

In [2]:
import numpy as np, os
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

#한글설정
import matplotlib.font_manager as fm

font_dirs = ['./dsml_data/nanum', ] #서버에서의 경로 /usr/share/fonts/truetype/nanum
font_files = fm.findSystemFonts(fontpaths=font_dirs)

for font_file in font_files:
    fm.fontManager.addfont(font_file)

# 한글 출력을 위해서 폰트 옵션을 설정
# "axes.unicode_minus" : 마이너스가 깨질 것을 방지

sns.set(font = "NanumBarunGothic",
        rc = {"axes.unicode_minus":False},
        style = 'darkgrid')


NameError: name 'fonr_file' is not defined

In [2]:
all_df = pd.read_csv('./dsml_data/df_all_best.csv')
df_d1d5 = pd.read_csv('./dsml_data/df_d1d5_best.csv')
df_d6d10 = pd.read_csv('./dsml_data/df_d6d10_best.csv')

In [3]:
all_df

,feature,baseline_bce,bce,diff
0,51006,1.014362,1.001220,-0.013142
1,63323026201,1.014362,1.006389,-0.007973
2,51277,1.014362,1.006995,-0.007367
3,0,1.014362,1.009931,-0.004431
4,55390000401,1.014362,1.010688,-0.003673
...,...,...,...,...
4063,10019055302,1.014362,1.041859,0.027498
4064,54817525,1.014362,1.043492,0.029130
4065,67434504,1.014362,1.046829,0.032467
4066,74606211,1.014362,1.051030,0.036668


# Method1: Combine Score for 방향성
```
ReLU(FI*) - sin(FI**)

FI*: diff
FI**: mean(preds2) - mean(preds)


preds를 위해 LSH의 05-2 참고
```

In [4]:
# 1) DATA 
import random    
# ---------------------
seed_num = 42
# ---------------------
random.seed(seed_num)

x = np.load('./dsml_data/x_(7727,10,4068).npy')
y = np.load('./dsml_data/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

In [9]:
# 2) MODEL
from tensorflow.keras.models import load_model
lstm2 = load_model('./dsml_data/model/(LSTM_best_4068)seed42-05-0.5029.hdf5')

In [1]:
# 3)
# FEATURES
a = pd.read_csv('./dsml_data/total_data_7727.csv')
features = list(a['ITEMID'].sort_values().unique()) # 4068개 ITEMID 

results = []
preds = lstm2.predict(X_test)
preds

# # COMPUTE BASELINE
# from tensorflow.keras.losses import BinaryCrossentropy
# bce = BinaryCrossentropy()
# baseline_bce = bce(y_test, preds).numpy()
# results.append({'feature':'BASELINE','baseline_bce':baseline_bce}) 

for k in tqdm(range(len(features))):

    save_col = X_test[:,:,k].copy()

    # REVERSE ALL FEATURE K
    X_test[:,:,k] = np.abs(X_test[:,:,k]-1)

    # preds2
    preds2 = lstm2.predict(X_test)

    preds2
    break
    X_test[:,:,k] = save_col

NameError: name 'pd' is not defined